In [1]:
!pip install simplemma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 MB 11.0 MB/s eta 0:00:00


In [2]:
from simplemma import text_lemmatizer
import pandas as pd
import numpy as np
import ast
import re

In [4]:

df = pd.read_excel("/content/yorum.xlsx")
etiketler = [0,1]
df = df[df.Durum.isin(etiketler)].copy() #sadece 1-0 olanlari kayit eder 2leri siler
df.to_excel('yorumlar.xlsx', index=False) # bu dosyaya
df.head()


,Metin,Durum
0,evet anlatıldığı gibi,1
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1
3,Daha öncede aynısını almıştım çok güzel ve kal...,1
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1


In [7]:
stop_words = None
with open("stopwords.txt", "r") as stop_file:
    stop_words = set(stop_file.read().splitlines())

In [11]:
def clean_text(text):
    text = text.replace("Â", "a")
    text = text.replace("â", "a")
    text = text.replace("î", "i")
    text = text.replace("Î", "ı")
    text = text.replace("İ", "i")
    text = text.replace("I", "ı")
    text = text.replace(u"\u00A0", " ")
    text = text.replace("|", " ")

    text = re.sub(r"@[A-Za-z0-9]+", " ", text)
    text = re.sub(r"(.)\1+", r"\1\1", text)
    text = re.sub(r"https?:\/\/\S+", " ", text)
    text = re.sub(r"http?:\/\/\S+", " ", text)
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"#(\w+)", " ", text)
    text = re.sub(r"^\x00-\x7F]+", " ", text)
    text = re.sub(r"[^A-Za-zâîığüşöçİĞÜŞÖÇ]+", " ", text)
    text = re.sub(r"((https://[^\s]+))", " ", text)

    text = " ".join(text.lower().strip().split())
    text = text_lemmatizer(text, lang="tr")

    return " ".join([word for word in text if word not in stop_words])


In [8]:
len(df)


13777

In [12]:

df["Metin"] = df["Metin"].astype(str) # AttributeError: 'float' object has no attribute 'replace' hatasi için
df["clean"] = df["Metin"].apply(clean_text)


In [15]:

df.to_excel('temizveri.xlsx', index=False)

In [14]:
import random
for i in range(3):
    rand_idx = random.randrange(len(df))
    ozet = df.iloc[rand_idx]["Metin"]
    print("Orijinal:")
    print(ozet + "\n")
    print("Temizlenmiş:")
    print(df.iloc[rand_idx]["clean"])
    print("/" * 50 + "\n")


Orijinal:
Herhangi bir pantolon ile kullanılmaz. Aşırı geniş

Temizlenmiş:
pantolon kullan aşırı geniş
//////////////////////////////////////////////////

Orijinal:
kırık geldi ve rengi de farklıydı

Temizlenmiş:
kırık gel renk fark
//////////////////////////////////////////////////

Orijinal:
Resmen çocukluğuma döndüm çok güzel ve küçük olduğunu yazmışlar yorumlarda oysa boyutu oldukça iyi

Temizlenmiş:
resmen çocukluk dön güzel küçük yaz yorum boyut oldukça iyi
//////////////////////////////////////////////////

